# Um breve tour do Fatiando a Terra

In [ ]:
import fatiando

In [ ]:
print(fatiando.__version__)

## Criando formas geométricas

In [ ]:
from fatiando import mesher

In [ ]:
prisma = mesher.Prism(1, 5, 9, 29, 0, 3)

In [ ]:
prisma

In [ ]:
prisma.get_bounds()

In [ ]:
prisma.addprop('density', 250)

In [ ]:
prisma.props

In [ ]:
from fatiando.vis import myv

In [ ]:
modelo = [prisma]

myv.figure()
myv.prisms(modelo)
myv.show()

In [ ]:
myv.figure()
myv.prisms(modelo, 'density')
bounds = [0, 20, 0, 30, 0, 20]
myv.outline(bounds)
myv.wall_north(bounds)
myv.show()

## Modelagem direta

In [ ]:
from fatiando import gridder

In [ ]:
x, y = gridder.regular(bounds[:4], (50, 50))

In [ ]:
x, y

In [ ]:
import numpy as np

In [ ]:
z = np.zeros_like(x)

In [ ]:
z

In [ ]:
from fatiando.gravmag import prism

In [ ]:
grav = prism.gz(x, y, z, modelo)

In [ ]:
grav

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.tricontourf(y, x, grav, 60)

## Dados grav

In [ ]:
lon, lat, y, x, alt, topo, grav = np.loadtxt('gravidade-havai.txt', unpack=True)

In [ ]:
plt.tricontourf(y, x, grav, 60)

In [ ]:
from fatiando.gravmag import normal_gravity

In [ ]:
disturbio = grav - normal_gravity.gamma_closed_form(lat, alt)

In [ ]:
plt.tricontourf(y, x, disturbio, 60)
plt.colorbar()

In [ ]:
bouguer = disturbio - normal_gravity.bouguer_plate(topo)

In [ ]:
plt.tricontourf(y, x, bouguer, 60)
plt.colorbar()

# Anomalia magnética

## Carregando os dados

In [ ]:
x, y, z, anom = np.loadtxt('anomalia-mag.txt', unpack=True)
shape = (50, 50)

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()

## Calcula o TGA

In [ ]:
from fatiando.gravmag import transform

In [ ]:
tga = transform.tga(x, y, anom, shape)

In [ ]:
plt.tricontourf(y, x, tga, 60, cmap='RdBu_r')
plt.colorbar()

In [ ]:
transform.

In [ ]:
dx = transform.derivx(x, y, anom, shape)
dy = transform.derivy(x, y, anom, shape)
dz = transform.derivz(x, y, anom, shape)

In [ ]:
plt.tricontourf(y, x, dz, 60, cmap='RdBu_r')
plt.colorbar()

## Localizando a fonte com deconvolução de Euler

In [ ]:
from fatiando.gravmag import EulerDeconv

In [ ]:
solver = EulerDeconv(x, y, z, anom, dx, dy, dz, structural_index=3)

In [ ]:
solver.fit()

In [ ]:
print(solver.estimate_)

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()
plt.plot(solver.estimate_[1], solver.estimate_[0], 'ok')

## Várias soluções usando janelas

In [ ]:
from fatiando.gravmag import EulerDeconvMW

In [ ]:
janela = EulerDeconvMW(x, y, z, anom, dx, dy, dz, 3, (10, 10), (3000, 3000))
janela.fit()

In [ ]:
janela.estimate_

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()
plt.plot(janela.estimate_[:, 1], janela.estimate_[:, 0], 'ok')

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.scatter(janela.estimate_[:, 1], janela.estimate_[:, 0], s=100, c=janela.estimate_[:, 2], cmap="Greens")
plt.colorbar()

In [ ]:
from fatiando.vis import myv

In [ ]:
myv.figure()
myv.points(janela.estimate_, size=50, color=(1, 0, 0))
bounds = [0, 8000, 0, 10000, 0, 1000]
myv.axes(myv.outline(bounds))
myv.wall_north(bounds)
myv.wall_bottom(bounds)
myv.show()

## Redução ao polo

In [ ]:
inc, dec = -19.5, -18.5

In [ ]:
polo = transform.reduce_to_pole(x, y, anom, shape, inc, dec, inc, dec)

In [ ]:
plt.tricontourf(y, x, polo, 60, cmap='RdBu_r')
plt.colorbar()

## Estimando a direção de magnetização

In [ ]:
from fatiando.gravmag import magdir

In [ ]:
solver = EulerDeconv(x, y, z, anom, dx, dy, dz, structural_index=3).fit()

In [ ]:
direcao = magdir.DipoleMagDir(x, y, z, anom, inc, dec, [solver.estimate_])

In [ ]:
direcao.fit()

In [ ]:
direcao.estimate_

In [ ]:
sinc, sdec = direcao.estimate_[0][1:]

In [ ]:
polo = transform.reduce_to_pole(x, y, anom, shape, inc, dec, sinc, sdec)

In [ ]:
plt.tricontourf(y, x, polo, 60, cmap='RdBu_r')
plt.colorbar()